## Generating point cloud

In [1]:
from pyodm import Node

In [2]:
n = Node('localhost', 3000)

In [3]:
from os import listdir

In [5]:
prefix = '../images/original'
files = [prefix + '/' + f for f in listdir(prefix)]

In [7]:
task = n.create_task(files, {'dsm': True})

In [8]:
task.wait_for_completion()

In [9]:
task.download_assets("/media/antonio/Ubuntail/Dropbox/education/phd/courses/processamento_imagens/assignments/final/pyodm")

'/media/antonio/Ubuntail/Dropbox/education/phd/courses/processamento_imagens/assignments/final/pyodm'

## Visualizind point cloud

In [21]:
import laspy
import open3d as o3d
import numpy as np
from os import listdir

In [2]:
pc_data_dir = '/media/antonio/Ubuntail/Dropbox/education/phd/courses/processamento_imagens/assignments/final/pyodm/entwine_pointcloud/ept-data/'

In [55]:
points = None
for fname in listdir(pc_data_dir):
    pc = laspy.read(pc_data_dir + fname)
    point_data = np.stack([pc.X, pc.Y, pc.Z], axis=0).transpose((1, 0))
    if points is None:
        points = point_data
    else:
        points = np.concatenate([points, point_data])
        

In [66]:
idxs = np.where((points[:, 0] <= 1000) & ((points[:, 1] <= 1000)))
print(idxs)
surface = o3d.geometry.PointCloud()
surface.points = o3d.utility.Vector3dVector(points[idxs])
o3d.visualization.draw_geometries([surface])


(array([], dtype=int64),)


In [27]:
import matplotlib.pyplot as plt

In [28]:
def show(img, plt_size=(16, 10), cmap=None):
    fig = plt.figure(figsize=plt_size)
    plt.imshow(img, cmap=cmap if cmap is not None else (None if len(img.shape) == 3 else 'gray'))

In [32]:
img = surface.Image

In [34]:
img.RGBDImage

<Type.RGBDImage: 9>

In [36]:
import cv2

In [72]:
len(points)

10771891

In [75]:
def points_to_image(points, canvas):
    resolution = canvas.shape
    x_range = np.max(points[:, 0]) - np.min(points[:, 0])
    y_range = np.max(points[:, 1]) - np.min(points[:, 1])
    h_step = y_range // resolution[1]
    w_step = x_range // resolution[0]
    for h in range(resolution[0]):
        for w in range(resolution[1]):
            w_head = w_step * w
            w_tail = w_head + w_step
            h_head = h_step * h
            h_tail = h_head + h_step
            idxs = np.where(  (points[:, 0] >= w_head)
                            & (points[:, 0] < w_tail)
                            & (points[:, 1] >= h_head)
                            & (points[:, 1] < h_tail))
            region = points[idxs]
            canvas[h, w] = np.mean(region[:, 2])
            
    
    

In [76]:
img = np.zeros((500, 625))

In [77]:
points_to_image(points, img)

/home/antonio/dev/pyvenvs/pointcloud/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/antonio/dev/pyvenvs/pointcloud/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

In [68]:
np.argmin(points, axis=0)

array([  48883, 5449935, 5449935])

In [71]:
points[5449935]

array([146262,      0,      0], dtype=int32)

In [74]:
img = points_to_image(points[:, 0], points[:, 1], points[:, 2], (500, 625))

ValueError: invalid entry in coordinates array